In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
import librosa

%matplotlib inline


In [2]:
# Calculate SNR 
def calculateSNR(st, st_h):
    st_sum = np.sum(np.abs(st))
    diff_sum = np.sum(np.abs(st-st_h))
    return 10*math.log(st_sum**2/diff_sum**2)

In [37]:
# Read data

def read_data(path, prefix, files_to_load):
    signal = np.zeros((513, 0))
    noise = np.zeros((513, 0))
    mixture = np.zeros((513, 0))
    for i in range(files_to_load):
        s, _=librosa.load('{}/{}s{:04d}.wav'.format(path, prefix, i), sr=None)
        S=librosa.stft(s, n_fft=1024, hop_length=512)
        signal = np.hstack((signal, S))
        s, _=librosa.load('{}/{}n{:04d}.wav'.format(path, prefix, i), sr=None)
        S=librosa.stft(s, n_fft=1024, hop_length=512)
        noise = np.hstack((noise, S))
        s, _=librosa.load('{}/{}x{:04d}.wav'.format(path, prefix, i), sr=None)
        S=librosa.stft(s, n_fft=1024, hop_length=512)
        mixture = np.hstack((mixture , S))

    return signal, noise, mixture



In [40]:
# TRAIN_DATA_LEN = 1199
TRAIN_DATA_LEN = 50
signal, noise, mixture = read_data('timit-homework/tr', 'tr', TRAIN_DATA_LEN)
v_signal, v_noise, v_mixture = read_data('timit-homework/v', 'v', 50)

In [41]:
print(signal.shape)
print(v_signal.shape)
# Try to output~
# _, sr = librosa.load('timit-homework/tr/trs0000.wav', sr=None)
# librosa.output.write_wav("./test_concat.wav", librosa.istft(signal, hop_length=512), sr)

(513, 4310)
(513, 4310)


In [8]:
# Get abs of training data
signal_abs = np.abs(signal)
noise_abs = np.abs(noise)
mixture_abs = np.abs(mixture)

In [10]:
# short hand for common used 
signal_T = signal.T
noise_T = noise.T
mixture_T = mixture.T
signal_abs_T = signal_abs.T
noise_abs_T = noise_abs.T
mixture_abs_T = mixture_abs.T

In [ ]:
# 

In [14]:
# Calculate IBM 
M = np.zeros_like(signal_abs_T)
(ht, wd) = M.shape

for i in range(ht):
    for j in range(wd):
        if signal_abs_T[i][j] > noise_abs_T[i][j]:
            M[i][j] = 1


In [17]:
# Sanity check the shape of X(mixture) and M
print(mixture_abs_T.shape, M.shape)

(4310, 513) (4310, 513)


In [83]:

# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128

# Network Parameters
num_input = 513   # total data input (513 channel)
time_steps = 8   #
num_hidden = 128  # hidden layer num of features
num_classes = 513 # total classes (513 channel)

is_training = False

In [84]:
with tf.variable_scope("rnn_model", reuse=tf.AUTO_REUSE):
    X = tf.placeholder("float", [None, time_steps, num_input])
    Y = tf.placeholder("float", [None, num_classes])

# batch_size*128*513
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}

biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


In [167]:
# tf.reset_default_graph()

def RNN(x, weights, biases):    
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)
    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, axis=1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_hidden)
    # Get lstm cell output
    outputs, states = tf.nn.static_rnn(lstm_cell, x, dtype=tf.float32)
    outputs = [ tf.matmul(output, weights['out']) + biases['out'] for output in outputs]
    stack_outputs = tf.stack(outputs)

    return tf.reshape(stack_outputs, (-1, num_classes))


In [168]:
# Calculate loss, train_op
with tf.variable_scope("rnn_model", reuse=tf.AUTO_REUSE):
    Y_pred = RNN(X, weights, biases)
    loss = tf.losses.mean_squared_error(labels=Y, predictions=Y_pred)
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [170]:
print(Y_pred.shape, Y.shape)

(?, 513) (?, 513)


In [171]:
train_data = tf.data.Dataset.from_tensor_slices(tf.constant(mixture_abs_T))
label_data = tf.data.Dataset.from_tensor_slices(tf.constant(M.reshape(-1, num_classes)))
batch_data = tf.data.Dataset.zip((train_data, label_data)).repeat().batch(batch_size)

iterator = batch_data.make_one_shot_iterator()
next_batch = iterator.get_next()

In [172]:
# Initial session
sess=tf.Session()
tf.global_variables_initializer().run(session=sess)

In [173]:
# Create checkpoint
saver = tf.train.Saver()
CHECK_POINT_FILE_NAME = "./hw3_2.ckpt"

try:
    saver.restore(sess, CHECK_POINT_FILE_NAME)
except:
    pass

In [175]:
# Training
# for i in range(training_steps+1):
for i in range(1000):
    (batch_x, batch_y) = sess.run(next_batch)
    batch_x = batch_x.reshape(-1, time_steps, num_input)
#     print(batch_x.shape, batch_y.shape)
    _, loss_value = sess.run((train_op, loss), feed_dict={X: batch_x, Y: batch_y})
    if i % 200 == 0:
        is_training = False
        print('Training Step:' + str(i) + '  Accuracy =  ' + 
              str(sess.run(accuracy, feed_dict={x: mixture_abs_T, y_: mnist.test.labels})) + 
              '  Loss =  ' + str(loss_value))
        is_training = True

        

(16, 8, 513) (128, 513)
(16, 8, 513) (128, 513)
